# Machine learning and deep learning methods applied to predicting customer status

Purpose: Predict active and not active clients based on the proposed data structure and naive data structure. Here, we implemented Support Vector Machines (SVM), Random Forest (RF), K-nearest Neighbours (KNN) and Lasso.

Author: Gabriel Rodrigues Palma and Rafael de Andrade Moral

# Packages used in the project

In [1]:
# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import LeakyReLU

# Machine learning packages
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import LeaveOneOut

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Functions used in the project

In [2]:
def create_dataset(path, response_class):
    ''' This function reads and prepare the datasets for applying the ML and DL methods'''
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0', 'Subject'])
    explanatory_variables = data.drop(columns = response_class)
    response_variables = data[response_class]        
    binarizer = LabelBinarizer()
    response_variables = binarizer.fit_transform(response_variables)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    hot_encode_response_variable = onehot_encoder.fit_transform(np.array(response_variables).reshape(-1, 1))          
    
    return(explanatory_variables, 
           hot_encode_response_variable, response_variables)

def check_zero_division_and_get_rates(cm):
    ''' This functions checks for divisions per zeros in the computation of 
       True and false positive rate based on the confusion matrix array. Also, 
       this function returns the checked rates'''
    
    if any(np.sum(cm, axis = 1)==0):            
            tpr = cm[:,1][1]/(np.sum(cm, axis = 1)[1]+1e-16)
            fpr = cm[:,1][0]/(np.sum(cm, axis = 1)[0]+1e-16)    
    else:           
        rates = cm[:,1]/np.sum(cm, axis = 1)    
        fpr = rates[0]    
        tpr = rates[1]
    if np.isnan(tpr):        
        tpr=0
        
    return(tpr, fpr)

def check_and_compute_rates(predictions, 
                            classes, 
                            cm):    
    ''' This function returns the values of true and false positive rate for special cases
       where the division is not possible to obtain automatically based on the confusion
       matrix array provided by sklearn'''
        
    if (sum(predictions) == 0 and sum(classes) == 0):        
        fpr = 0
        tpr = 0
    elif (sum(predictions) == len(predictions) and sum(classes) == len(classes)):        
        fpr = 0
        tpr = 1
    else:            
        tpr, fpr = check_zero_division_and_get_rates(cm)
        
    return(tpr, fpr)
    
def get_rates(y_pred,
              y_true):
    ''' This function get the true and false positive rates based on the 
       predictied'''
            
    cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
    tpr, fpr = check_and_compute_rates(y_pred, y_true, cm)
        
    return(tpr, fpr)

def get_rates_by_cross_validation(raw_data):
    ''' This function obtains the accuracy, true and false positive rates 
       based on the cross k-fold cross validation'''
            
    for train_index, test_index in KFold(n_splits=5, shuffle=True).split(patterns):
        
        x_train, x_test = patterns[train_index], patterns[test_index]
        y_train, y_test = classes[train_index], classes[test_index]
                
        pbp_predictions = pbp_prediction(patterns_array=x_test, 
                                         clustered_patterns = clustered_patterns, 
                                         d_base = d_base, alpha = alpha, 
                   outbreak_p_means = prediction.obtain_p_means_with_distance,
                   outbreak_prediction = prediction.predict_with_distance)
        rates = get_rates(predictions = pbp_predictions, classes = y_test)
        tpr.append(rates[0])
        fpr.append(rates[1])
    
    return(np.mean(tpr[tpr!=np.nan]), np.mean(fpr[fpr!=np.nan]))

def get_statistics(y_pred, y_true):
    ''' This function obtains the following statistics: Accuracy, True positive rate and 
       False positive rate'''

    auc = roc_auc_score(y_score = y_pred,
              y_true = y_true)
    acc = accuracy_score(y_pred = y_pred,
                         y_true = y_true)
    rates = get_rates(y_pred = y_pred,
              y_true = y_true)
    tpr = rates[0]
    fpr = rates[1]
    return([acc, tpr, fpr, auc])

# def get_methods_performance(X_train, X_test, 
#                             y_train, y_test, 
#                             raw_data):
#     ''' This function obtain the performance of each selected model'''
#     kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
#     # Support Vector Machine -----
#     ## Non-linear
#     nonlinear_svm = svm.NuSVC(gamma="auto")
#     # Create the parameter space
#     params = {"kernel": ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
#               "gamma": ['auto', 'scale'],
#              "degree": np.arange(1, 10, 1)}
#     nonlinear_svm_cv = RandomizedSearchCV(nonlinear_svm, params, cv=kf)
#     nonlinear_svm_cv.fit(X_train, y_train)
#     nonlinear_svm_predictions = nonlinear_svm_cv.predict(X_test)
    
#     ## Polinomial
#     polinomial_svm = svm.SVC(kernel='poly', degree = 3)
#     polinomial_svm.fit(X_train, y_train)
#     polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
#     # Random forest -----
#     rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
#     rf.fit(X_train, y_train)
#     random_forest_predictions = rf.predict(X_test)
    
#     # KNN -----
#     knn = KNeighborsClassifier(n_neighbors=1)
#     knn.fit(X_train, y_train)
#     knn_predictions = knn.predict(X_test)
    
#     # Lasso -----
#     lasso = Lasso(alpha=0.02)
#     lasso.fit(X_train, y_train)
#     lasso_predictions = np.round(lasso.predict(X_test))
    
#     # Deep Neural Network -----
# #     if raw_data==True:
# #         dnn = keras.models.load_model('DNN/DNN_model_params.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     elif raw_data == 'combined':
# #         dnn = keras.models.load_model('DNN/DNN_model_fulldata.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     else:    
# #         dnn = keras.models.load_model('DNN/DNN_model_hmm.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))

    
#     # Obtaining statistics -----
#     polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
#                                         y_true = y_test)
#     nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
#                                        y_true = y_test)
#     random_forest_performance = get_statistics(y_pred = random_forest_predictions,
#                                        y_true = y_test)
#     knn_performance = get_statistics(y_pred = knn_predictions,
#                                        y_true = y_test)    
#     lasso_performance = get_statistics(y_pred = lasso_predictions,
#                                        y_true = y_test)
# #     dnn_performance = get_statistics(y_pred = dnn_predictions,
# #                                        y_true = y_test)

#     return(polynomial_svm_performance, nonlinear_svm_performance, 
#            random_forest_performance, knn_performance, 
#            lasso_performance)
    
# def get_results_data(explanatory_variables,                             
#                      response_variables,                                                         
#                      test_size, raw_data):
#     ''' This function obtain the performance of the ML and DL methods based on
#        the prediction of client status'''
#     X_train, X_test, y_train, y_test = train_test_split(explanatory_variables, 
#                                                         response_variables, 
#                                                         test_size = test_size, 
#                                                         random_state = 42)
   
#     # Obtaining methods performance
#     polynomial_svm_performance, nonlinear_svm_performance, \
#     random_forest_performance, knn_performance, \
#     lasso_performance = get_methods_performance(X_train, X_test, 
#                                               y_train, y_test, 
#                                               raw_data)
        
    
#     model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
#                                                             'True Positive Rate', 
#                                                             'False Positive Rate', 
#                                                             'AUROC'],
#                                               'Polinomial SVM': [polynomial_svm_performance[0], 
#                                                                  polynomial_svm_performance[1], 
#                                                                  polynomial_svm_performance[2],
#                                                                 polynomial_svm_performance[3]], 
#                                               'Non linear SVM': [nonlinear_svm_performance[0], 
#                                                                  nonlinear_svm_performance[1], 
#                                                                  nonlinear_svm_performance[2], 
#                                                                 nonlinear_svm_performance[3]], 
#                                               'Random Forest': [random_forest_performance[0], 
#                                                                 random_forest_performance[1], 
#                                                                 random_forest_performance[2], 
#                                                                random_forest_performance[3]], 
#                                               'KNN': [knn_performance[0], 
#                                                       knn_performance[1], 
#                                                       knn_performance[2], 
#                                                       knn_performance[3]],                                                
#                                               'Lasso': [lasso_performance[0], 
#                                                         lasso_performance[1], 
#                                                         lasso_performance[2], 
#                                                         lasso_performance[3]]})

#     return(model_outputs_performance)
    

In [3]:
def get_methods_predictions(X_train, X_test, 
                            y_train, y_test, 
                            data_type):
    ''' This function obtain the performance of each selected model'''
    kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    # Support Vector Machine -----
    ## Non-linear
    nonlinear_svm = svm.NuSVC()
    
    nonlinear_svm.fit(X_train, y_train)
    
    nonlinear_svm_predictions = nonlinear_svm.predict(X_test)
    
    ## Polinomial
    polinomial_svm = svm.SVC(kernel='poly', degree = 3)
    polinomial_svm.fit(X_train, y_train)
    polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
    # Random forest -----
    rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
    rf.fit(X_train, y_train)
    random_forest_predictions = rf.predict(X_test)
    
    # KNN -----
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)
    knn_predictions = knn.predict(X_test)
    
    # Lasso -----
    lasso = Lasso(alpha=0.02)
    lasso.fit(X_train, y_train)
    lasso_predictions = np.round(lasso.predict(X_test))
    
    # Deep Neural Network -----
    if data_type == 'All':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(50,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
    elif data_type == 'm2':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(13,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))

    elif data_type == 'm3':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(16,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
    elif data_type == 'm4':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(19,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
        
    else:    
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(22,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
    

    return(polinomial_svm_predictions, nonlinear_svm_predictions, 
           random_forest_predictions, knn_predictions, 
           lasso_predictions, dnn_predictions)
def get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                                random_forest_predictions, knn_predictions, 
                                lasso_predictions, dnn_predictions, 
                                reponse_variable):
    
    '''This function obtain the methods performance based on their predictions for the leave one out cross valition'''
    polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
                                    y_true = reponse_variable)
    nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
                                       y_true = reponse_variable)
    random_forest_performance = get_statistics(y_pred = random_forest_predictions,
                                       y_true = reponse_variable)
    knn_performance = get_statistics(y_pred = knn_predictions,
                                       y_true = reponse_variable)    
    lasso_performance = get_statistics(y_pred = lasso_predictions,
                                       y_true = reponse_variable)
    dnn_performance = get_statistics(y_pred = dnn_predictions,
                                        y_true = reponse_variable)
    return(polynomial_svm_performance, nonlinear_svm_performance, 
           random_forest_performance, knn_performance, 
           lasso_performance, dnn_performance)

    
def get_results_data_loo(explanatory_variables,                         
                         response_variables,                                                         
                         data_type):
    ''' This function obtain the performance of the ML and DL methods based on
       the prediction of client status based on leave one out cross validation'''

    loo = LeaveOneOut()
    loo.get_n_splits(explanatory_variables)
    polinomial_svm_predictions, nonlinear_svm_predictions, \
           random_forest_predictions, knn_predictions, \
           lasso_predictions, dnn_predictions = ([], [], [], [], [], [])
    
    for train_index, test_index in loo.split(explanatory_variables):
        X_train, X_test = explanatory_variables[train_index], explanatory_variables[test_index]
        y_train, y_test = response_variables[train_index], response_variables[test_index]                
             
        ## Correcting errors related to column vector (n_observations, ) to (, n_observations)
        y_train = y_train.ravel() 
        y_test = y_test.ravel()        
    
        polynomial_svm_prediction, nonlinear_svm_prediction, \
        random_forest_prediction, knn_prediction, \
        lasso_prediction, dnn_prediction = get_methods_predictions(X_train, X_test, 
                                                   y_train, y_test, 
                                                   data_type)
        
        polinomial_svm_predictions.append(polynomial_svm_prediction[0])
        nonlinear_svm_predictions.append(nonlinear_svm_prediction[0])
        random_forest_predictions.append(random_forest_prediction[0])
        knn_predictions.append(knn_prediction[0])        
        lasso_predictions.append(lasso_prediction[0]) 
        dnn_predictions.append(dnn_prediction[0])
 
    polynomial_svm_performance, nonlinear_svm_performance, \
       random_forest_performance, knn_performance, \
       lasso_performance, dnn_performance = get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                            random_forest_predictions, knn_predictions, 
                            lasso_predictions, dnn_predictions, 
                            list(response_variables.flatten()))
    
    model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
                                                            'True Positive Rate', 
                                                            'False Positive Rate', 
                                                            'AUROC'],
                                              'Polinomial SVM': [polynomial_svm_performance[0], 
                                                                 polynomial_svm_performance[1], 
                                                                 polynomial_svm_performance[2],
                                                                polynomial_svm_performance[3]], 
                                              'Non linear SVM': [nonlinear_svm_performance[0], 
                                                                 nonlinear_svm_performance[1], 
                                                                 nonlinear_svm_performance[2], 
                                                                nonlinear_svm_performance[3]], 
                                              'Random Forest': [random_forest_performance[0], 
                                                                random_forest_performance[1], 
                                                                random_forest_performance[2], 
                                                               random_forest_performance[3]], 
                                              'KNN': [knn_performance[0], 
                                                      knn_performance[1], 
                                                      knn_performance[2], 
                                                      knn_performance[3]],                                                
                                              'Lasso': [lasso_performance[0], 
                                                        lasso_performance[1], 
                                                        lasso_performance[2], 
                                                        lasso_performance[3]], 
                                             'DNN': [dnn_performance[0], 
                                                        dnn_performance[1], 
                                                        dnn_performance[2], 
                                                        dnn_performance[3]]})

    return(model_outputs_performance)

# Importing Datasets

Before obtaining the performance of the learning algorithms selected for this paper, we need to import the datasets related to the Hidden Markov Models features and the peak features.

## Raw Midline EEG data Trial 1

In [4]:
all_raw_explanatory_variables_t1, \
  all_raw_hot_encode_response_variable_t1, all_raw_response_variable_t1 = create_dataset(path = '../../output_data/New_data/CoordinatesPlusEEGData/RawMidlinePlusCoordinateTrial1.csv', 
                                                                                        response_class = 'Class')

In [5]:
all_raw_explanatory_variables_t1.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5,dis_sum,angle_sum,average_speed,idle_time
0,1.085978e-07,-1.050929e-07,8.294140e-07,-5.906837e-10,0.868,0.604,0.129,0.074,0.515900,0.484100,...,0.025,0.180843,0.240996,0.243103,0.144828,0.190230,4.007659,651.069516,0.001659,4.50
1,1.080854e-07,-1.045971e-07,8.527908e-07,-6.459901e-10,0.590,0.899,0.096,0.184,0.530876,0.469124,...,0.030,0.156503,0.214782,0.282709,0.138557,0.207449,4.350488,1406.028426,0.002660,1.25
2,-6.478640e-08,6.269551e-08,6.327935e-07,-5.413374e-10,0.709,0.466,0.127,0.065,0.404959,0.595041,...,0.018,0.229291,0.181818,0.179896,0.210456,0.198539,4.195562,882.929882,0.002113,2.50
3,-1.573635e-09,1.522849e-09,6.501585e-07,-4.326145e-10,0.699,0.481,0.089,0.067,0.440678,0.559322,...,0.044,0.190293,0.208398,0.214176,0.224769,0.162365,4.203073,1916.215396,0.002636,2.25
4,2.001960e-07,-1.937350e-07,9.778229e-07,-7.200449e-10,0.722,1.087,0.105,0.164,0.627565,0.372435,...,0.027,0.237708,0.252226,0.111692,0.255517,0.142857,3.978881,2061.484914,0.002336,3.75


## Raw Midline EEG data Trial 12

In [6]:
all_raw_explanatory_variables_t12, \
  all_raw_hot_encode_response_variable_t12, all_raw_response_variable_t12 = create_dataset(path = '../../output_data/New_data/CoordinatesPlusEEGData/RawMidlinePlusCoordinateTrial12.csv', 
                                                                                           response_class = 'Class')

In [7]:
all_raw_explanatory_variables_t12.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5,dis_sum,angle_sum,average_speed,idle_time
0,1.156881e-07,-1.077369e-07,9.165390e-07,-7.693143e-10,0.959,0.677,0.146,0.095,0.315457,0.684543,...,0.133,0.194532,0.219769,0.256572,0.170347,0.158780,0.610205,225.982551,0.007746,0.75
1,9.669676e-08,-9.005086e-08,8.743374e-07,-7.271431e-10,0.620,1.054,0.058,0.297,0.590447,0.409553,...,0.305,0.149390,0.257114,0.183943,0.295732,0.113821,0.753753,240.335305,0.009937,0.50
2,-4.570386e-08,4.256267e-08,5.965999e-07,-4.592086e-10,0.425,0.694,0.044,0.180,0.470644,0.529356,...,0.182,0.129735,0.188447,0.133523,0.357008,0.191288,0.496671,93.845292,0.003795,2.75
3,1.880012e-08,-1.750800e-08,7.036014e-07,-5.416457e-10,0.787,0.513,0.118,0.062,0.423729,0.576271,...,0.095,0.095450,0.329170,0.116860,0.266726,0.191793,0.614849,1046.380417,0.007977,1.25
4,1.369923e-07,-1.275770e-07,9.263932e-07,-7.455344e-10,1.006,0.643,0.159,0.118,0.412071,0.587929,...,0.136,0.262227,0.234131,0.123829,0.138398,0.241415,0.762090,56.845578,0.010055,0.50


# Obtaining learning algorithms performance based on Leave one out cross validation

Now, we are obtaining accuracy, trus and false positive rates for Polinomial and no linear Support Vector Machines, Random Forests, K-neirest neighbors and Lasso learning algorithms.

## Raw Midline EEG data Trial 1

For the raw Midline EEG data, we are going to analise train and analyse the variables in the following way:

    1. All the features;
    2. The features for HMM with 2 parameters;
    3. The features for HMM with 3 parameters;
    4. The features for HMM with 4 parameters;
    5. The features for HMM with 5 parameters;

### All the features

In [9]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t1),                                                  
                     response_variables = all_raw_response_variable_t1,                                                                           
                     data_type = 'All')

2023-04-05 17:26:19.662018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 57ms/step


2023-04-05 17:26:24.619473: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:26:25.570785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 87ms/step


2023-04-05 17:26:30.585788: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:26:31.865362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 93ms/step


2023-04-05 17:30:27.217192: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:30:28.430347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 103ms/step


2023-04-05 17:30:34.030265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.410256,0.615385,0.461538,0.666667,0.564103
1,True Positive Rate,1.000000,0.318182,0.727273,0.545455,0.772727,1.000000
2,False Positive Rate,1.000000,0.470588,0.529412,0.647059,0.470588,1.000000
3,AUROC,0.500000,0.423797,0.598930,0.449198,0.651070,0.500000


### The features for HMM with 2 parameters

In [10]:
selected_columns_m2 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm2_mean.1', 
                    'hmm2_mean.2', 'hmm2_sd.1', 'hmm2_sd.2', 
                    'hmm2_p1', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m2_array = [elem in selected_columns_m2 for elem in all_raw_explanatory_variables_t1.columns]

In [11]:
all_raw_explanatory_variables_t1_m2 = all_raw_explanatory_variables_t1[selected_columns_m2]

In [12]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t1_m2),                                                  
                     response_variables = all_raw_response_variable_t1,                                                                           
                     data_type = 'm2')

2023-04-05 17:30:35.119959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 88ms/step


2023-04-05 17:30:40.811230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:30:41.755129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 113ms/step


2023-04-05 17:30:47.284306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:30:48.682179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 139ms/step


2023-04-05 17:35:27.223860: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:35:28.604089: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 147ms/step


2023-04-05 17:35:35.312532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.435897,0.717949,0.461538,0.461538,0.564103
1,True Positive Rate,1.000000,0.454545,0.772727,0.545455,0.636364,1.000000
2,False Positive Rate,1.000000,0.588235,0.352941,0.647059,0.764706,1.000000
3,AUROC,0.500000,0.433155,0.709893,0.449198,0.435829,0.500000


### The features for HMM with 3 parameters

In [13]:
selected_columns_m3 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm3_mean.1', 'hmm3_mean.2', 
                    'hmm3_mean.3', 'hmm3_sd.1', 
                    'hmm3_sd.2', 'hmm3_sd.3', 'hmm3_p1',
                    'hmm3_p2', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m3_array = [elem in selected_columns_m3 for elem in all_raw_explanatory_variables_t1.columns]

In [14]:
all_raw_explanatory_variables_t1_m3 = all_raw_explanatory_variables_t1[selected_columns_m3]

In [15]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t1_m3),                                                  
                     response_variables = all_raw_response_variable_t1,                                                                           
                     data_type = 'm3')

2023-04-05 17:35:36.546764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 152ms/step


2023-04-05 17:35:43.360371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:35:44.892276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 145ms/step


2023-04-05 17:35:51.651817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:35:52.706105: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 219ms/step


2023-04-05 17:41:26.177457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:41:27.294810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 178ms/step


2023-04-05 17:41:34.913320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.512821,0.692308,0.461538,0.358974,0.564103
1,True Positive Rate,1.000000,0.363636,0.681818,0.545455,0.500000,1.000000
2,False Positive Rate,1.000000,0.294118,0.294118,0.647059,0.823529,1.000000
3,AUROC,0.500000,0.534759,0.693850,0.449198,0.338235,0.500000


### The features for HMM with 4 parameters

In [16]:
selected_columns_m4 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm4_mean.1', 'hmm4_mean.2', 'hmm4_mean.3',
                       'hmm4_mean.4', 'hmm4_sd.1', 'hmm4_sd.2', 'hmm4_sd.3', 'hmm4_sd.4',
                       'hmm4_p1', 'hmm4_p2', 'hmm4_p3', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m4_array = [elem in selected_columns_m3 for elem in all_raw_explanatory_variables_t1.columns]

In [17]:
all_raw_explanatory_variables_t1_m4 = all_raw_explanatory_variables_t1[selected_columns_m4]

In [18]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t1_m4),                                               
                     response_variables = all_raw_response_variable_t1,                                                                           
                     data_type = 'm4')

2023-04-05 17:41:36.827148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 217ms/step


2023-04-05 17:41:44.820805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:41:46.289567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 205ms/step


2023-04-05 17:41:54.049616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:41:55.183697: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 196ms/step


2023-04-05 17:47:28.479211: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:47:29.949726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 202ms/step


2023-04-05 17:47:38.378634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.410256,0.692308,0.461538,0.666667,0.564103
1,True Positive Rate,1.000000,0.363636,0.727273,0.545455,0.772727,1.000000
2,False Positive Rate,1.000000,0.529412,0.352941,0.647059,0.470588,1.000000
3,AUROC,0.500000,0.417112,0.687166,0.449198,0.651070,0.500000


### The features for HMM with 5 parameters

In [19]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_raw_explanatory_variables_t1.columns]

In [20]:
all_raw_explanatory_variables_t1_m5 = all_raw_explanatory_variables_t1[selected_columns_m5]

In [21]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t1_m5),                                                  
                     response_variables = all_raw_response_variable_t1,                                                                           
                     data_type = 'm5')

2023-04-05 17:47:39.783382: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:47:48.331816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 250ms/step


2023-04-05 17:47:49.891472: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 207ms/step


2023-04-05 17:47:58.746372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:47:59.865787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-05 17:54:12.145557: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:54:24.339650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 366ms/step


2023-04-05 17:54:26.052030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 17:54:38.534332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 426ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.564103,0.666667,0.461538,0.589744,0.564103
1,True Positive Rate,1.000000,0.545455,0.772727,0.545455,0.727273,1.000000
2,False Positive Rate,1.000000,0.411765,0.470588,0.647059,0.588235,1.000000
3,AUROC,0.500000,0.566845,0.651070,0.449198,0.569519,0.500000


## Raw Midline EEG data Trial 12

For the Minmax scale EEG data, we are going to analise train and analyse the variables in the following way:

    1. All the features;
    2. The features for HMM with 2 parameters;
    3. The features for HMM with 3 parameters;
    4. The features for HMM with 4 parameters;
    5. The features for HMM with 5 parameters;

### All the features

In [23]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t12),                                                  
                     response_variables = all_raw_response_variable_t12,                                                                           
                     data_type = 'All')

2023-04-05 21:40:45.853415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:41:02.741937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 540ms/step


2023-04-05 21:41:04.247179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:41:17.295964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 383ms/step


2023-04-05 21:41:18.577432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:41:31.400911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-05 21:49:45.605999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:50:02.476858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 541ms/step


2023-04-05 21:50:04.490823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:50:17.974704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 424ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.538462,0.538462,0.564103,0.564103,0.717949,0.564103
1,True Positive Rate,0.954545,0.681818,0.727273,0.545455,0.818182,1.000000
2,False Positive Rate,1.000000,0.647059,0.647059,0.411765,0.411765,1.000000
3,AUROC,0.477273,0.517380,0.540107,0.566845,0.703209,0.500000


### The features for HMM with 2 parameters

In [24]:
selected_columns_m2 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm2_mean.1', 
                    'hmm2_mean.2', 'hmm2_sd.1', 'hmm2_sd.2', 
                    'hmm2_p1', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m2_array = [elem in selected_columns_m2 for elem in all_raw_explanatory_variables_t12.columns]

In [25]:
all_raw_explanatory_variables_t12_m2 = all_raw_explanatory_variables_t12[selected_columns_m2]

In [26]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t12_m2),                                                 
                     response_variables = all_raw_response_variable_t12,                                                                           
                     data_type = 'm2')

2023-04-05 21:50:19.675261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:50:33.358206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 443ms/step


2023-04-05 21:50:34.706219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:50:48.248305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 415ms/step


2023-04-05 21:50:50.041325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 21:51:04.069590: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-05 22:01:31.703061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:01:48.489189: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 570ms/step


2023-04-05 22:01:50.590945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:02:07.217396: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 561ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.538462,0.589744,0.717949,0.564103,0.743590,0.538462
1,True Positive Rate,0.954545,0.727273,0.772727,0.545455,0.863636,0.954545
2,False Positive Rate,1.000000,0.588235,0.352941,0.411765,0.411765,1.000000
3,AUROC,0.477273,0.569519,0.709893,0.566845,0.725936,0.477273


### The features for HMM with 3 parameters

In [27]:
selected_columns_m3 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm3_mean.1', 'hmm3_mean.2', 
                    'hmm3_mean.3', 'hmm3_sd.1', 
                    'hmm3_sd.2', 'hmm3_sd.3', 'hmm3_p1',
                    'hmm3_p2', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m3_array = [elem in selected_columns_m3 for elem in all_raw_explanatory_variables_t12.columns]

In [28]:
all_raw_explanatory_variables_t12_m3 = all_raw_explanatory_variables_t12[selected_columns_m3]

In [29]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t12_m3),                                                
                     response_variables = all_raw_response_variable_t12,                                                                           
                     data_type = 'm3')

2023-04-05 22:02:09.066494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:02:26.521764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 573ms/step


2023-04-05 22:02:28.551769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:02:45.743572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 610ms/step


2023-04-05 22:02:47.268216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:03:04.198975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-05 22:13:29.825683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:13:45.289347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 476ms/step


2023-04-05 22:13:46.641483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:14:01.699405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 513ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.538462,0.615385,0.692308,0.564103,0.769231,0.589744
1,True Positive Rate,0.954545,0.818182,0.681818,0.545455,0.863636,0.772727
2,False Positive Rate,1.000000,0.647059,0.294118,0.411765,0.352941,0.647059
3,AUROC,0.477273,0.585561,0.693850,0.566845,0.755348,0.562834


### The features for HMM with 4 parameters

In [30]:
selected_columns_m4 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm4_mean.1', 'hmm4_mean.2', 'hmm4_mean.3',
                       'hmm4_mean.4', 'hmm4_sd.1', 'hmm4_sd.2', 'hmm4_sd.3', 'hmm4_sd.4',
                       'hmm4_p1', 'hmm4_p2', 'hmm4_p3', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m4_array = [elem in selected_columns_m4 for elem in all_raw_explanatory_variables_t12.columns]

In [31]:
all_raw_explanatory_variables_t12_m4 = all_raw_explanatory_variables_t12[selected_columns_m4]

In [32]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t12_m4),                                                  
                     response_variables = all_raw_response_variable_t12,                                                                           
                     data_type = 'm4')

2023-04-05 22:14:04.011856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:14:19.950159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 527ms/step


2023-04-05 22:14:21.386514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:14:36.612284: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 541ms/step


2023-04-05 22:14:38.480634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:14:53.839710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-05 22:25:11.363306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:25:27.539507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 555ms/step


2023-04-05 22:25:28.985629: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-05 22:25:45.797665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.538462,0.589744,0.743590,0.564103,0.717949,0.564103
1,True Positive Rate,0.954545,0.772727,0.772727,0.545455,0.818182,1.000000
2,False Positive Rate,1.000000,0.647059,0.294118,0.411765,0.411765,1.000000
3,AUROC,0.477273,0.562834,0.739305,0.566845,0.703209,0.500000


### The features for HMM with 5 parameters

In [8]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_raw_explanatory_variables_t12.columns]

In [9]:
all_raw_explanatory_variables_t12_m5 = all_raw_explanatory_variables_t12[selected_columns_m5]

In [10]:
get_results_data_loo(explanatory_variables = np.array(all_raw_explanatory_variables_t12_m5),                                                  
                     response_variables = all_raw_response_variable_t12,                                                                           
                     data_type = 'm5')

Metal device set to: Apple M1


2023-04-06 05:48:09.718688: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-06 05:48:09.718950: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-06 05:48:09.976214: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-06 05:48:10.552274: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 94ms/step


2023-04-06 05:48:15.353208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 05:48:16.338779: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 59ms/step


...

1/1 [==============================] - 0s 100ms/step


2023-04-06 05:52:13.569201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 05:52:14.834286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 107ms/step


2023-04-06 05:52:20.337945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.538462,0.615385,0.692308,0.564103,0.743590,0.512821
1,True Positive Rate,0.954545,0.681818,0.818182,0.545455,0.863636,0.909091
2,False Positive Rate,1.000000,0.470588,0.470588,0.411765,0.411765,1.000000
3,AUROC,0.477273,0.605615,0.673797,0.566845,0.725936,0.454545
